Copyright 2021 National Technology & Engineering Solutions<br>
of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,<br>
the U.S. Government retains certain rights in this software.<br>
<br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>
<br>
   http://www.apache.org/licenses/LICENSE-2.0<br>
<br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()
    secret_share = protocol.uniform(shape=(2,2))
    secret = protocol.encoder.decode(protocol.reveal(secret_share))
    log.info(f"Player {communicator.rank} secret: {secret}")
    ltz_share = protocol.less_than_zero(secret_share)
    ltz = protocol.reveal(ltz_share)
    log.info(f"Player {communicator.rank} less_than_zero: \n{ltz}")
    not_ltz_share = protocol.logical_not(ltz_share)
    not_ltz = protocol.reveal(not_ltz_share)
    log.info(f"Player {communicator.rank} not_less_than_zero: \n{not_ltz}")

In this example, we will generate a small matrix of uniform random field elements which, in the context of our encoding, will be comprised of (probably) both negative and positive values.

We then call the less_than_zero function on these values which will return boolean values in the field. Then we logically invert these values. Note that logical_not is well defined only on boolean operands. Calling logical_not on values that are not either 0 or 1, will likely have unexpected results. 

See below that the results of less_than_zero are indeed logically negated in not_less_than_zero.

In [7]:
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:58149 from tcp://127.0.0.1:58149.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:58149 from tcp://127.0.0.1:58150.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:58149 from tcp://127.0.0.1:58151.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 0 secret: [[6.76824778e+13 5.36390378e+13]
 [1.22349172e+14 7.83189011e+13]]
INFO:root:Player 1 secret: [[6.76824778e+13 5.36390378e+13]
 [1.22349172e+14 7.83189011e+13]]
INFO:root:Player 2 secret: [[6.76824778e+13 5.36390378e+13]
 [1.22349172e+14 7.83189011e+13]]
INFO:root:Player 0 less_than_zero: 
[[0 0]
 [0 0]]
INFO:root:Player 1 less_than_zero: 
[[0 0]
 [0 0]]
INFO:root:Player 2 less_than_zero: 
[[0 0]
 [0 0]]
INFO:root:Player 0 not_less_than_zero: 


[None, None, None]